In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! unzip  /content/drive/MyDrive/nycu2023mlfinalproject.zip -d ./data

In [ ]:
import glob
import os
import csv
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Data transformations
data_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# Loading the dataset (using the full dataset for training)
dataset = ImageFolder(root='/content/data/data/train', transform=data_transforms)
folder = '/content/data/data/test1/test'
file_list = glob.glob(os.path.join(folder, '*'))


# Load the model
model = models.resnet152(pretrained=True)  # Make sure this matches your training configuration
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, len(dataset.classes))  # Replace with the number of classes in your dataset
)
model.load_state_dict(torch.load('/content/drive/MyDrive/final.pth'))  # Load your saved model
model = model.to(device)
model.eval()

test_transform =  transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])


print(file_list)
predictions = []
file_names = []
for file in file_list :
  image = Image.open(file)
  image = test_transform(image)
  image = image.unsqueeze(0).to(device)

  filename= os.path.basename(file)[:-4]
  with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        print(file)
        print(predicted)
        print(dataset.classes[predicted])

        file_names.append(filename)
        predictions.append(dataset.classes[predicted.item()])


   # Save predictions to CSV file
  with open('/content/drive/MyDrive/submission.csv', mode='w', newline='') as file:
      writer = csv.writer(file)
      writer.writerow(['id', 'label'])
      for i in range(len(file_names)):
          writer.writerow([file_names[i], predictions[i]])

  print("Predictions saved to submission.csv")




